In [1]:
import openpyxl as opl
#import related to tkinter file dialog
from tkinter import *
from tkinter import ttk
from tkinter import filedialog as fd
import pandas as pd
import os
import math
import datetime
import shutil
import numpy as np
#logg;ing related info
import logging
import xlsxwriter

In [2]:
"""This Files Saves a list of Data Frame"""
def saveDataFrame(dataframes,filepath,names):
    #writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    #writer= pd.ExcelWriter(filepath)
    for df,sname in zip(dataframes,names):
        df.to_excel(filepath,sheet_name=sname)   
    #writer.save()  
"""This Files Writes Data Frame in Proper Place of a Excel Sheet"""
"""Functions for Transferring Data Frame to Autocad Input File"""
def writeDataFrameToExcel(data_frame,drow,dcol,wbook_name,wsheet_name):
    """
    Parameters:
    data_frame= name of the frame from which data to be transferred
    drow=row offset to be added to dframe row index
    dcol=column offset to be added to dframe column index
    wbook_name=name of the destination work book
    wsheet_name=name of the destination work sheet    
    """
    wb=opl.load_workbook(wbook_name)
    #print(wb.sheetnames)
    transfer_sheet=wb[wsheet_name]
    dfshape=data_frame.shape 
    print(dfshape)
    for i in range(0,dfshape[0]):
        row_index=i+drow
        for j in range(0,dfshape[1]):
            transfer_sheet.cell(row=row_index,column=j+dcol).value=data_frame.iloc[i,j]
    wb.save(wbook_name)

In [3]:
"""Sediment Transport Related Functions"""
def tau_sub_c_from_swammi_mittal_formula(d50):
    """
    this formula calcualtes critcal shear stress as per Mittal and Swamee formula
    given in "Irrigation Enginering and Hydraulic Structure" S.K.Garg 5th edition 
    page 101 equation 4.12   
    d50 median sediment size in mm
    
    """
    T1=math.sqrt(1+0.177*d50*d50)
    T2=0.409*d50*d50
    tauc=0.155+(T2/T1)
    print("for d50={}  tau_sub_c={}".format(d50,tauc))
    return round(tauc,3)
def shear_veloity(d,s):
    """this eqation calcualte shear velocity from flow depth and longsection slope
    d=depth of flow in m
    s=long section slope
    g=accelaration of gravity=9.81 m/s
    """
    g=9.81
    ustar=math.sqrt(g*d*s)
    return ustar
def findDepthForIncipientMotion(d50):
    """This function calcualte minimum depth of flow 
    required to for incipient motion
    d50=median sediment size in mm
    rho_w=density of water =1000 kg/m^3
    """
    rho_w=1000
    tau_c=tau_sub_c_from_swammi_mittal_formula(d50)
    ustra_critical=math.sqrt(tau_c/rho_w)
def findSheildFactor(d50):
    """
    This function calcualtes shield's factor from d50 and long slope.
    d50=median grain diamter in mm
    S=long slope of water
    rho_sed=sediment specific gravirty=2.65
    rho_water=specific gravity of water
    gamma_w=unit weight of water=9810 N/m^3
    """
    Ss=2.65
    tauc=tau_sub_c_from_swammi_mittal_formula(d50)
    gamma_w=9810
    T2=gamma_w*(d50/1000)*(Ss-1)
    theta=tauc/T2
    return theta
def bedShearStress(h,S):
    
    """
    This function calcualtes Bed Shear Stress/Tractive force    
    h=flow depth in m
    S=longitudial slope
    """
    gamma_w=9810
    tau=gamma_w*h*S
    return tau
    
def particleReynoldsNumber(h,S,d50):
    """
    This function calcualtes Particle Reynold's Number
    d50=median sediment size in mm
    h=flow depth in m
    S=longitudial slope
    rho_w=density of water=1000 kg/m^3
     neu=kinametic viscosity of water=10^-6
    """
    neu=math.pow(10,-6)
    tau=bedShearStress(h,S)
    ustar=math.sqrt(tau/1000)
    d_m=d50/1000
    Re=(ustar*d_m)/neu
    return Re
    
    
def DensimetricParticleFroudeNumber(d50,Ua):
    """
    this function calcualtes particle froude nember
    d50=grainsize in mm
    Ua=cross sectional average velocity in m/sc
    rwo_s=specific gravity of sediment=2.65
    row_w=specific gravity of water=1
    g=9.81 m/sec^2 
   
    """
    rho_s=2.65
    rho_w=1.0
    d_m=d50/1000
    g=9.81
    Term1=(rho_s-rho_w)/rho_w
    Term2=d_m*9.1
    Term3=math.sqrt(Term1*Term2)
    FD=Ua/Term3
    return FD
def transeVerseBedSlope(d50,Ua,da,rc):
    FD=DensimetricParticleFroudeNumber(d50,Ua)
    theta=findSheildFactor(d50)
    ST=4.8*math.sqrt(theta)*FD*(da/rc)
    return ST
def RoughnessReynoldsNumber(d50,ustar):
    """This function calcualtes Roughness Reynolds Number
        Which is used to determine flow is hydraulically smooth or Rough
        d50=bed particle size in mm
        nu=kinamatic viscosity of water
    """
    Yr=d50/1000
    nu=math.pow(10,-6)
    Reb=(ustar*Yr)/nu
    if Reb<5:
        y0=nu/(9*ustar)
    else:
        y0=Yr/30
    myvalues={"Reb":Reb,"y0":y0}
    return myvalues

In [4]:
"""Function to read data from excel and transfer to to proper sheet"""
def fromLetterToColumn(letter):
    letters=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    numbers=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
    letter=letter.strip()    
    #print("letter={} col={}".format(letter,local_value))
    letter_length=len(letter)
    if len(letter)==1:        
        i=letters.index(letter)
        local_value=numbers[i]
        col_value=local_value    
    elif len(letter)==2:
        i1=letters.index(letter[0])
        i2=letters.index(letter[1])
        letter_val1=numbers[i1]
        letter_val2=numbers[i2]
        print("L1={} L2={}".format(letter_val1,letter_val2))
        col_value= letter_val1*26+letter_val2
    else:
        col_value=-99
    return col_value
#this function corrects distance if survey is done right to left
def shiftDistance_for_xvalues(x0,xoriginal):
    x_shifted=[]
    for x in xoriginal:
        x_shifted.append(x0-x)
    return  x_shifted

def readSheetValue_Horizontal(sheet_name,r1,r2,c1,c2,chn,sec_no,direction):
    print("row1={} row2={} col1={} col2={}".format(r1,r2,c1,c2))
    row1=r1
    row2=r2
    col1=fromLetterToColumn(c1)
    col2=fromLetterToColumn(c2)
    print("row1={} row2={} col1={} col2={}".format(row1,row2,col1,col2))
    distance=[]
    rl=[]
    remarks=[]
    chaiange=[]
    section_no=[]
    for c in range(col1,col2+1):
        dist_value=sheet_name.cell(row=row1,column=c).value
        distance.append(dist_value)
        rl_value=sheet_name.cell(row=row1+1,column=c).value
        rl.append(rl_value)        
        rem=sheet_name.cell(row=row2,column=c).value
        if rem is None:
            rem1=""
        else:
            rem1=rem.strip()
        remarks.append(rem1)
        chaiange.append(chn)
        section_no.append(sec_no)
    #changing values as per direction
    if direction==2:
        x0=distance[-1]
        distance_shifted=shiftDistance_for_xvalues(x0,distance)
        distance_c=distance_shifted[::-1]
        rl_c=rl[::-1]
        remarks_c=remarks[::-1]
        myvalue={"dist":distance_c,"rl":rl_c,"remarks":remarks_c,"chainage":chaiange,"sectionNo":section_no} 
    else:
        myvalue={"dist":distance,"rl":rl,"remarks":remarks,"chainage":chaiange,"sectionNo":section_no}   
    logger.debug("\n\n\n\n\n")
    logger.debug(myvalue)
    logger.debug("\n\n\n\n\n")
    print(myvalue)
    return myvalue
    


In [5]:
def readSheetValue_Vertical(sheet_name,r1,r2,c1,c2,chn,sec_no,direction):
    print("row1={} row2={} col1={} col2={}".format(r1,r2,c1,c2))
    row1=r1
    row2=r2
    col1=fromLetterToColumn(c1)
    col3=fromLetterToColumn(c2)
    col2=col3-1
    print("row1={} row2={} col1={} col2={}".format(row1,row2,col1,col2))
    distance=[]
    rl=[]
    remarks=[]
    chaiange=[]
    section_no=[]
    for r in range(r1,r2+1):
        dist_value=sheet_name.cell(row=r,column=col1).value
        distance.append(dist_value)
        rl_value=sheet_name.cell(row=r,column=col2).value
        rl.append(rl_value)        
        rem=sheet_name.cell(row=r,column=col3).value
        if rem is None:
            rem1=""
        else:
            rem1=rem.strip()
        remarks.append(rem1)
        chaiange.append(chn)
        section_no.append(sec_no)
    #changing values as per direction
    #changing values as per direction
    if direction==2:
        x0=distance[-1]
        distance_shifted=shiftDistance_for_xvalues(x0,distance)
        distance_c=distance_shifted[::-1]
        rl_c=rl[::-1]
        remarks_c=remarks[::-1]
        myvalue={"dist":distance_c,"rl":rl_c,"remarks":remarks_c,"chainage":chaiange,"sectionNo":section_no} 
    else:
        myvalue={"dist":distance,"rl":rl,"remarks":remarks,"chainage":chaiange,"sectionNo":section_no}    
    logger.debug("\n\n\n\n\n")
    logger.debug(myvalue)
    logger.debug("\n\n\n\n\n")
    print(myvalue)
    return myvalue





    
   
    


In [6]:
"""Class for Creating Khal_Sheet_Index"""
class khal_sheet_Nos:
    def __init__(self,khal_Name,length,xsection_list,start_sheet_no,y0,dwg_base_no,dwg_date):
        self.Name=khal_Name
        self.y0=y0
        self.Index_map_sheet_No=start_sheet_no
        self.Titles=[]        
        self.Notes_Sheet_No=start_sheet_no+1
        no_long_section=math.ceil(length/3000.0)
        print("Total sheet for Long Section={}".format(no_long_section))
        self.Long_Section_Sheet_No=[start_sheet_no+1+i 
                                    for i in range(1,no_long_section+1)]
        print("LS Sheets={}".format(self.Long_Section_Sheet_No))
        k=self.Long_Section_Sheet_No[-1]
        cross_section_sheet_list=[]
        for index,elem in enumerate(xsection_list):
            if index%2==0:
                k=k+1
                cross_section_sheet_list.append(k)
            else:
                cross_section_sheet_list.append(k)
        data={"SectionName":xsection_list,"Sheet_No":cross_section_sheet_list}
        self.xsection_sheet_df=pd.DataFrame(data)
        self.Details=cross_section_sheet_list[-1]+1
        self.Next_Index=self.Details+1
        """Generating Titles for Drawing Sheet"""
        self.generateTitles()
        """Generating Sheet Coorinates"""
        n=len(self.Titles)
        self.Xcoord=[]
        self.Ycoord=[]
        self.width=[]
        self.DrawingNo=[]
        self.DrawingDate=[]
        self.DSSheetNo=[]
        #dwg_base_no,dwg_date
        for i in range(0,n):
            self.Xcoord.append(100000+i*10000)            
            self.Ycoord.append(self.y0)
            self.width.append(8000)
            if i+1 <=9:
                self.DrawingNo.append(dwg_base_no+"-0"+str(i+1)+"/"+str(n))
                self.DSSheetNo.append("0"+str(i+1)+"/"+str(n))
            else:
                self.DrawingNo.append(dwg_base_no+"-"+str(i+1)+"/"+str(n))
                self.DSSheetNo.append(str(i+1)+"/"+str(n))
            self.DrawingDate.append(dwg_date)
                
                
            
        
    def generateTitles(self):
        """generatig sheet Titles"""
        Titles=[]
        Titles.append("Index Map & Site Map for "+self.Name)
        Titles.append("Notes for "+self.Name)
        for x in self.Long_Section_Sheet_No:
            Titles.append("Long Section of "+self.Name)
        unique_xs_sheet=self.xsection_sheet_df.Sheet_No.unique()  
        for s in unique_xs_sheet:
            Titles.append("Typical Cross Section of "+self.Name)
        Titles.append("Details for "+self.Name)
        self.Titles=Titles
    
    
    
    def display(self):
        print("Khal Name={} Index Map={} Notes={} "              
              .format(self.Name,self.Index_map_sheet_No,self.Notes_Sheet_No))
        print("Long Sections={}".format(self.Long_Section_Sheet_No))
        print("Xsections={}".format(self.xsection_sheet_df))
        print("Details={}".format(self.Details))

"""Class for creating Khal Long Sections"""
class Khal_Long_Sections:
    def __init__(self,khal_name,xsection_list,index_df,LS_Code,head=None):
        self.xsection_codes=xsection_list
        if head:
            self.xsection_codes.insert(0,head)
        self.LS_codes=[LS_Code for xs in self.xsection_codes]
        data={"LSCode":self.LS_codes,"CSCode":self.xsection_codes}
        self.DataDetails=pd.DataFrame(data)

In [7]:
"""
This Class is used for calculation of arclenth parmeter of Section
"""
class arc_len_param:
    def __init__(self,xvalues,yvalues):
        self.xvalues=xvalues
        self.yvalues=yvalues        
        n=len(self.xvalues)
        self.delx=[0]*n
        self.dely=[0]*n
        self.dels=[0]*n
        self.arclength=[0]*n
        self.tangent_i=[0]*n
        self.tangent_j=[0]*n
        self.normal_i=[0]*n
        self.normal_j=[0]*n
        self.theta=[0]*n
        self.slope_ratio=[0]*n
        self.segment_number=[0]*n
        self.calculateArcParameter()
    def calculateArcParameter(self):
        n=len(self.xvalues)
        total_length=0
        for i in range(1,n):
            delx=self.xvalues[i]-self.xvalues[i-1]
            dely=self.yvalues[i]-self.yvalues[i-1]
            dels=math.sqrt(delx*delx+dely*dely)
            total_length=total_length+dels
            print("x={} y={} delx={} dely={} dels={} TL={}".format(self.xvalues[i],self.yvalues[i],
                                                                   delx,dely,dels,total_length))
            ti=delx/dels
            tj=dely/dels
            if ti !=0:
                theta=math.atan(tj/ti)
                
            else:
                theta=0
            if dely!=0:
                m=delx/dely
            else:
                m=111111
            self.delx[i]=delx
            self.dely[i]=dely
            self.dels[i]=dels
            self.tangent_i[i]=ti
            self.tangent_j[i]=tj
            self.normal_i[i]=tj*-1
            self.normal_j[i]=ti            
            self.slope_ratio[i]=m
            self.segment_number[i]=i  
            self.arclength[i]= total_length
            self.theta[i]=theta
            
    def get_arc_len_df2(self):
        
        data={"segment_no": self.segment_number,
            "x":self.xvalues,"y":self.yvalues,'arc_length':self.arclength,'delx': self.delx,
              'dely':self.dely,'Angle':self.theta,'slope': self.slope_ratio,
              'ti':self.tangent_i,'tj':self.tangent_j,'ni':self.normal_i,'nj':self.normal_j
             }
        arc_len_df=pd.DataFrame(data)
        return arc_len_df
    
        
        
    
    def get_arc_len_df(self):        
        data={"segment_no": self.segment_number,
            "x":self.xvalues,"y":self.yvalues,'arc_length':self.arclength,'delx': self.delx,
              'dely':self.dely,'Angle':self.theta,'slope': self.slope_ratio,
              'ti':self.tangent_i,'tj':self.tangent_j,'ni':self.normal_i,'nj':self.normal_j
             }
        arc_len_df=pd.DataFrame(data)
        return arc_len_df

In [8]:
"""This class is used for Calcualtion of Xsection Propteries"""
class xesc_sec_param:
    def __init__(self,xvalues,yvalues,d50,S,sec_no):

        
        #logger.debug("#####################Values in constructor #############################")
        #logger.debug("xvalues for section={}".format(xvalues))
        #logger.debug("yvalues for section={}".format(yvalues))
        #logger.debug("#######################################################################")           
        self.xvalues=xvalues
        self.yvalues=yvalues
         
        self.sec_no=sec_no
        n=len(self.xvalues)        
        self.lowestRL()
        self.d50=d50
        self.S=S
        self.displaySec()
    def findleftIndex(self,lwl):
        for index, value in enumerate(self.yvalues):
            #print("index={} value={}".format(index,value))
            if value<lwl:
                return index
    def findRightIndex(self,lwl):
        n=len(self.yvalues)
        for i in range(1,n):
            if self.yvalues[i*-1]<lwl:
                return n-i
    def findLeftInterSection(self,index,lwl):
        y1=self.yvalues[index-1]
        y2=self.yvalues[index]
        x1=self.xvalues[index-1]
        x2=self.xvalues[index]
        slope=x1-x2/y1-y2
        x=x2+(lwl-y2)*slope
        return x
    def findRightInterSection(self,index,lwl):
        n=len(self.yvalues)-1        
        y1=self.yvalues[index]
        y2=self.yvalues[index+1]
        x1=self.xvalues[index]
        x2=self.xvalues[index+1]        
        slope=(x2-x1)/(y2-y1)
        #print("x1={} x2={} y1={} y2={} slope={}".format(x1,x2,y1,y2,slope))
        x=x1+(lwl-y1)*slope
       
            
        return x
    def calcualteAreaBelowLWL(self,lwl):
        i1=self.findleftIndex(lwl)
        i2=self.findRightIndex(lwl)
        logger.debug("si={} fin={}".format(i1,i2))
        yval1=list(self.yvalues[i1:i2+1])
        xval1=list(self.xvalues[i1:i2+1])         
        x0=round(self.findLeftInterSection(i1,lwl),2)
        xlast=round(self.findRightInterSection(i2,lwl),2)
        #print("left intersection={} right intersection={}".format(x0,xlast))
        #print("before any insertion............")
        #print(xval1)
        xval1.insert(0,x0)
        yval1.insert(0,lwl)
        xval1.append(xlast)
        yval1.append(lwl)
        logger.debug("distance={}".format(xval1))
        logger.debug("RL={}".format(yval1))
        logger.debug("water level={}".format(lwl))
        area=0
        n1=len(xval1)
        delx=[]
        avg_y=[]
        max_y=lwl-min(yval1)
        for i in range(1,n1):
            delx.append( xval1[i]-xval1[i-1])
            h1=lwl-yval1[i]
            h2=lwl-yval1[i-1]
            dely_avg=(h1+h2)*0.5
            avg_y.append(dely_avg)
            a=dely_avg*(xval1[i]-xval1[i-1])
            if (a>=0):
                area=area+a  
            
        #print("after  insertion...........")
        #print(xval1)
        #print(yval1)
        #print(delx)       
        top_width= xval1[-1]-xval1[0]
        logger.debug("Left={} Right={} Topwidth={}".format(xval1[-1],xval1[0],top_width))
        avg_h=area/top_width
        if (avg_h<0):
            avg_h=avg_h*-1
        #print("area={} topwidth={} avg_h={} max_depth={}".format(area,top_width,avg_h,max_y))
        myreturn_values={"area":area,"avg_h":avg_h,"top_width":top_width,'max_y':max_y}
        return myreturn_values
    def lowestRL(self):
        y_min_sec=min(self.yvalues)
        #logger.debug("minimum depth={}".format( y_min_sec))
        i=self.yvalues.index(y_min_sec)
        left_sec_y_max=max(self.yvalues[0:i])
        right_sec_y_max=max(self.yvalues[i:-1])
        sec_max_yvalues=min([left_sec_y_max,right_sec_y_max])
        self.useable_hwl=sec_max_yvalues
        self.ymin_sec=y_min_sec
    def calcualteSedimentSectionProperties(self):
        area=[]
        elevation=[]
        flow_depth=[]
        top_width=[]
        avg_depth=[]
        shield_factor=[]
        densimetric_paticle_reynold_number=[]
        h1=self.ymin_sec
        h2=self.useable_hwl
        n=int((h2-h1)/0.05)
        tau_c=[]
        prt_reynolds_number=[]
        ustar=[]
        tau=[]
        sheild_factor_theta=[]
        Reb=[]
        y0=[]
        multiplier=[]
        Ua=[]
        Q_Sediment_transport=[]
        Q_mannings=[]
        discharge_factor=[]
        
        
        myrange=[h1+i*0.05 for i in range(2,n)]
        for h in myrange:
            myvalues=self.calcualteAreaBelowLWL(h)
            elevation.append(h)
            flow_depth.append(h-h1)
            current_area=myvalues["area"]
            area.append(current_area)
            da=myvalues["avg_h"]
            avg_depth.append(da)
            top_width.append(myvalues["top_width"])
            """Calcualting Sediment Transport Parameter"""
            tau_c_current=tau_sub_c_from_swammi_mittal_formula(self.d50)
            tau_c.append(tau_c_current)   
            print("h={} s={} d50={}  ".format(da,self.S,d50))
            prt_reynolds_number_current=particleReynoldsNumber(da,self.S,self.d50)           
            prt_reynolds_number.append(prt_reynolds_number_current)
            ustar_current=shear_veloity(da,self.S)
            ustar.append(ustar_current)
            tau_cuurent=bedShearStress(da,self.S)
            tau.append(tau_cuurent)
            sheild_factor_theta_current=findSheildFactor(self.d50)            
            sheild_factor_theta.append(sheild_factor_theta_current)
            myvalues=RoughnessReynoldsNumber(self.d50,ustar_current)
            Reb_current=myvalues["Reb"]
            y0_current=myvalues["y0"]
            Reb.append(Reb_current)
            y0.append(y0_current)
            nu=math.pow(10,-6)
           
            y0_current_corrected=nu/(9*ustar_current)
            current_multiplier=math.log(da/y0_current)-1
            current_velocity=2.50*ustar_current*current_multiplier
            current_sed_discharge=current_velocity*current_area
            multiplier.append(current_multiplier)
            Ua.append(current_velocity)
            Q_Sediment_transport.append(current_sed_discharge)
            Q_maanings_current=(1/0.026)*current_area*math.pow(da,2/3)*math.pow(self.S,0.5)
            Q_mannings.append( Q_maanings_current)
            discharge_factor.append(current_sed_discharge/Q_maanings_current)
        data={"WL":elevation,"Depth of flow":flow_depth,"Wetted Area":area,
              "Top Width":top_width,"da":avg_depth,'tau_c':tau_c,
              "particle_reynold's_number":prt_reynolds_number,"u*":ustar,"tau":tau,
              'sheild_factor_theta':sheild_factor_theta,"Reb":Reb,"y0":y0,
              "ln(Yw/y0)-1":multiplier,"Ua":Ua,"Q_from_sed_param":Q_Sediment_transport,
              "Q_from_mannings":Q_mannings,"Q_S/Q_M":discharge_factor
             }
        mydata_frame=pd.DataFrame(data)
        return mydata_frame
    def displaySec(self):
        #self.xvalues=xvalues
        #self.yvalues=yvalues
        #self.sec_no=sec_no
        n=len(self.xvalues)
        logger.debug("########################Displaying Section Properties###########################")
        logger.debug("x={}".format( self.xvalues))
        logger.debug("y={}".format(self.yvalues))
        logger.debug("section no={} total data points={}".format(self.sec_no,n))
        logger.debug("deepest point of section={}".format(self.ymin_sec))
        logger.debug("Maximum Water Level without over flow={}".format(self.useable_hwl))
        
        pass

In [9]:
"""Chainage and RL Calculation related function"""
def calculteStartChainage(sec_chainage):
    start_chn=[]
    start_chn.append(sec_chainage[0]/1000)
    n=len(sec_chainage)
    for i in range(1,n):
        chn=(sec_chainage[i-1]+sec_chainage[i])/1000
        start_chn.append(chn*0.5)
    return start_chn
def calculteEndChainage(sec_chainage):
    end_chn=[]    
    n=len(sec_chainage)
    for i in range(0,n-1):
        chn=(sec_chainage[i]+sec_chainage[i+1])/1000
        end_chn.append(chn*0.5)
    end_chn.append(sec_chainage[n-1]/1000)
    return end_chn
    
def calcualteRLFromChainage(out_fall_rl,slope,chn_list):
    """
    out_fall_rl in m-PWD
    slope in cm/km
    chn_list  chainage list in km    
    """
    myrl=[]
    out_fall_dist=chn_list[-1]
    for ch in chn_list:
        rl=out_fall_rl+(ch)*(slope/100)
        rl=round(rl,2)
        myrl.append(rl)
    return myrl
    
def calcualteRLFromChainage(out_fall_rl,slope,chn_list):
    """
    out_fall_rl in m-PWD
    slope in cm/km
    chn_list  chainage list in km    
    """
    myrl=[]
    out_fall_dist=chn_list[-1]
    for ch in chn_list:
        rl=out_fall_rl+(ch)*(slope/100)
        rl=round(rl,2)
        myrl.append(rl)
    return myrl



def calcualteRLFromChainageFromUS(out_fall_rl,slope,chn_list):
    """
    out_fall_rl in m-PWD
    slope in cm/km
    chn_list  chainage list in km    
    """
    myrl=[]
    out_fall_dist=chn_list[-1]
    for ch in chn_list:
        rl=out_fall_rl+( out_fall_dist-ch)*(slope/100)
        rl=round(rl,2)
        myrl.append(rl)
    return myrl
def calcualte_cl_X(sec_list,combined_df):
    logger.debug("\n\n\nDetailed Debugging of Calcualtion of CL\n\n\n\n")
    #logger.debug(combined_df)
    cl_x=[]
    lbank_rl=[]
    rabnk_rl=[]
    cl_rl=[]
    sec_loc=[]
    ls_chaiange=[]
    k=1
    for sec in sec_list:
        logger.debug("\n\n\n\nInitiated Section no ={}**********************************".format(sec))        
        mydf=combined_df[combined_df['Section No']==sec]
        logger.debug(mydf)
        mydf2=mydf[mydf['Remarks']=="CL"]  
        #logger.debug("data for cross section={}".format(sec))
        logger.debug(mydf2)
        dist=mydf2.iloc[0,2]
        
        mydf3=mydf[mydf['Remarks']=="LB"]
        logger.debug(mydf3)
        lbank_rl.append(mydf3.iloc[0,3])
        mydf4=mydf[mydf['Remarks']=="RB"]
        logger.debug(mydf4)
        rabnk_rl.append(mydf4.iloc[0,3])
        logger.debug("sencionno={} CL distance={} RL={}".format(sec,dist,mydf2.iloc[0,3]))
        cl_rl.append(mydf2.iloc[0,3])
        cl_x.append(dist)
        ls_chaiange.append(mydf2.iloc[0,1])
        if k%2==1:
            myloc="A"
        else:
            myloc="B"
        sec_loc.append(myloc)
        k=k+1
        logger.debug("Finished Section no ={}**********************************".format(sec))
    myvalues={"cl_x":cl_x,"cl_rl": cl_rl,"lbank_rl":lbank_rl,"rabnk_rl":rabnk_rl,
            "sec_loc":sec_loc,"ls_chaiange":ls_chaiange }
    return myvalues
def calcualte_cl_X2(sec_list,combined_df):
    logger.debug("\n\n\nDetailed Debugging of Calcualtion of CL\n\n\n\n")
    logger.debug(combined_df)
    cl_x=[]
    lbank_rl=[]
    rabnk_rl=[]
    cl_rl=[]
    sec_loc=[]
    ls_chaiange=[]
    k=1
    for sec in sec_list:
        mydf=combined_df[combined_df['Section No']==sec]
        mydf2=mydf[mydf['Remarks']=="CL"]  
        #logger.debug("data for cross section={}".format(sec))
        #logger.debug(mydf2)
        dist=mydf2.iloc[0,2]
        logger.debug(mydf3)
        mydf3=mydf[mydf['Remarks']=="LB"] 
        logger.debug(mydf3)
        lbank_rl.append(mydf3.iloc[0,3])
        
        mydf4=mydf[mydf['Remarks']=="RB"]
        logger.debug(mydf4)
        rabnk_rl.append(mydf4.iloc[0,3])
        logger.debug("sencionno={} CL distance={} RL={}".format(sec,dist,mydf2.iloc[0,3]))
        cl_rl.append(mydf2.iloc[0,3])
        cl_x.append(dist)
        ls_chaiange.append(mydf2.iloc[0,1])
        if k%2==1:
            myloc="A"
        else:
            myloc="B"
        sec_loc.append(myloc)
        k=k+1
    myvalues={"cl_x":cl_x,"cl_rl": cl_rl,"lbank_rl":lbank_rl,"rabnk_rl":rabnk_rl,
            "sec_loc":sec_loc,"ls_chaiange":ls_chaiange }
    return myvalues
"""Calculating Data Sheet for Drawing Sheet"""
def calcualteDrawingSheetData(khal_name,dwg_no,drawing_date):
    mydata=index_df[index_df['KhalName']==khal_name]
    xsec_list=list(mydata['XsectionNo'])
    chn_list=list(mydata['chainage'])
    print(xsec_list)
    print(chn_list)
    khal_length=chn_list[-1]-chn_list[0]
    #print("length of Khal={}".format( khal_length))
    long_sec_sheet_no=math.ceil(khal_length/5000)
    n=len(xsec_list)
    cros_sec_sheet_no=math.ceil(n/2)
    print("length of Khal={} Long_sec_sheet={}".format( khal_length,long_sec_sheet_no))
    print("total cross section={} sheet_no={}".format(n,cros_sec_sheet_no))
    #Title	Design No	Date	x0	y0	width	sheet_no    
    Title=[]
    DesignNo=[]
    design_date=[]
    x0=[]
    y0=[]
    width=[]
    sheet_no=[]
    """populating sheet title"""
    Title.append("Index Map & Site Map for "+khal_name)
    Title.append("Notes")    
    for i in range(0,long_sec_sheet_no):
        Title.append("Long Section of "+khal_name )
    for i in range(0, cros_sec_sheet_no):
        
        if (i+1%2!=0):            
            Title.append("Typical Cross Section of "+ khal_name)
    Title.append("Details")
    print(Title)
    n=len(Title)
    for i in range(0,n):
        if i<10:
            my_design_no=dwg_no+"-0"+str(i+1)+"/"+str(n)
        else:
            my_design_no=dwg_no+"-"+str(i+1)+"/"+str(n)
        DesignNo.append(my_design_no)
    print(DesignNo)
    design_date=[str(drawing_date) for x in Title]
    print(design_date)
    for i in range(1,n+1):
        sheet_no.append(str(i)+" of "+str(n))
        x0.append(100000+(i-1)*10000)
        y0.append(0)
        width.append(8000)
    print(sheet_no)
    myvalues={"Title":Title,"DesignNo":DesignNo,"design_date":design_date,
              "x0":x0,"y0":y0,"width":width,"sheet_no":sheet_no}
    return myvalues

In [10]:
"""Function to read data from excel and transfer to to proper sheet"""
def fromLetterToColumn(letter):
    letters=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    numbers=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
    letter=letter.strip()    
    #print("letter={} col={}".format(letter,local_value))
    letter_length=len(letter)
    if len(letter)==1:        
        i=letters.index(letter)
        local_value=numbers[i]
        col_value=local_value    
    elif len(letter)==2:
        i1=letters.index(letter[0])
        i2=letters.index(letter[1])
        letter_val1=numbers[i1]
        letter_val2=numbers[i2]
        print("L1={} L2={}".format(letter_val1,letter_val2))
        col_value= letter_val1*26+letter_val2
    else:
        col_value=-99
    return col_value


def readSheetValue2(sheet_name,r1,r2,c1,c2,chn,sec_no):
    print("row1={} row2={} col1={} col2={}".format(r1,r2,c1,c2))
    row1=r1
    row2=r2
    col1=fromLetterToColumn(c1)
    col2=fromLetterToColumn(c2)
    print("row1={} row2={} col1={} col2={}".format(row1,row2,col1,col2))
    distance=[]
    rl=[]
    remarks=[]
    chaiange=[]
    section_no=[]
    for c in range(col1,col2+1):
        dist_value=sheet_name.cell(row=row1,column=c).value
        distance.append(dist_value)
        rl_value=sheet_name.cell(row=row1+1,column=c).value
        rl.append(rl_value)        
        rem=sheet_name.cell(row=row2,column=c).value
        if rem is None:
            rem1=""
        else:
            rem1=rem.strip()
        remarks.append(rem1)
        chaiange.append(chn)
        section_no.append(sec_no)
    myvalue={"dist":distance,"rl":rl,"remarks":remarks,"chainage":chaiange,"sectionNo":section_no}    
    print(myvalue)
    return myvalue

    
def readAndTransfer(data_sheet,index_df,transfer_sheet):    
    print("I am in proper function")

In [11]:
"""Class for Creating Khal_Sheet_Index"""
class khal_sheet_Nos:
    def __init__(self,khal_Name,length,xsection_list,start_sheet_no,y0,dwg_base_no,dwg_date):
        self.Name=khal_Name
        self.y0=y0
        self.Index_map_sheet_No=start_sheet_no
        self.Titles=[]        
        self.Notes_Sheet_No=start_sheet_no+1
        no_long_section=math.ceil(length/3000.0)
        print("Total sheet for Long Section={}".format(no_long_section))
        self.Long_Section_Sheet_No=[start_sheet_no+1+i 
                                    for i in range(1,no_long_section+1)]
        print("LS Sheets={}".format(self.Long_Section_Sheet_No))
        k=self.Long_Section_Sheet_No[-1]
        cross_section_sheet_list=[]
        for index,elem in enumerate(xsection_list):
            if index%2==0:
                k=k+1
                cross_section_sheet_list.append(k)
            else:
                cross_section_sheet_list.append(k)
        data={"SectionName":xsection_list,"Sheet_No":cross_section_sheet_list}
        self.xsection_sheet_df=pd.DataFrame(data)
        self.Details=cross_section_sheet_list[-1]+1
        self.Next_Index=self.Details+1
        """Generating Titles for Drawing Sheet"""
        self.generateTitles()
        """Generating Sheet Coorinates"""
        n=len(self.Titles)
        self.Xcoord=[]
        self.Ycoord=[]
        self.width=[]
        self.DrawingNo=[]
        self.DrawingDate=[]
        self.DSSheetNo=[]
        #dwg_base_no,dwg_date
        for i in range(0,n):
            self.Xcoord.append(100000+i*10000)            
            self.Ycoord.append(self.y0)
            self.width.append(8000)
            if i+1 <=9:
                self.DrawingNo.append(dwg_base_no+"-0"+str(i+1)+"/"+str(n))
                self.DSSheetNo.append("0"+str(i+1)+"/"+str(n))
            else:
                self.DrawingNo.append(dwg_base_no+"-"+str(i+1)+"/"+str(n))
                self.DSSheetNo.append(str(i+1)+"/"+str(n))
            self.DrawingDate.append(dwg_date)
                
                
            
        
    def generateTitles(self):
        """generatig sheet Titles"""
        Titles=[]
        Titles.append("Index Map & Site Map for "+self.Name)
        Titles.append("Notes for "+self.Name)
        for x in self.Long_Section_Sheet_No:
            Titles.append("Long Section of "+self.Name)
        unique_xs_sheet=self.xsection_sheet_df.Sheet_No.unique()  
        for s in unique_xs_sheet:
            Titles.append("Typical Cross Section of "+self.Name)
        Titles.append("Details for "+self.Name)
        self.Titles=Titles
    
    
    
    def display(self):
        print("Khal Name={} Index Map={} Notes={} "              
              .format(self.Name,self.Index_map_sheet_No,self.Notes_Sheet_No))
        print("Long Sections={}".format(self.Long_Section_Sheet_No))
        print("Xsections={}".format(self.xsection_sheet_df))
        print("Details={}".format(self.Details))
"""Class for creating Khal Long Sections"""
class Khal_Long_Sections:
    def __init__(self,khal_name,xsection_list,index_df,LS_Code,head=None):
        self.xsection_codes=xsection_list
        if head:
            self.xsection_codes.insert(0,head)
        self.LS_codes=[LS_Code for xs in self.xsection_codes]
        data={"LSCode":self.LS_codes,"CSCode":self.xsection_codes}
        self.DataDetails=pd.DataFrame(data)

In [12]:
"""This Function Checks Length of Arrays in Dictionary.Which is necessary for Pandas Data Frame Creation """
def check_array_length_for_df(data_dict):
    for key,values in data_dict.items():
        print("list {} has lenngth={}".format(key,len(values)))

In [13]:
"""
#############################################################################################################################
Programme Execution Begins Here 
#############################################################################################################################

"""
#create an instance of tkinter frame or windowroot=Tk()
#create an instance of tkinter frame or window
root=Tk()
root.geometry("750x150")
work_book_path=fd.askopenfilename(title="select Excel Input File")
root.withdraw()
#print(work_book_path)
dir_name=os.path.dirname(work_book_path)
head,tail=os.path.split(work_book_path)
print(dir_name)
print(tail)
file_names=tail.split(".")
print(dir_name)
out_path=os.path.join(dir_name,"Data_Prep_"+file_names[0]+"."+file_names[1])
out_path2=os.path.join(dir_name,"data_agg_output.xlsx")

D:/Design_All_DC_5/AEMostofa/Southwest Project/SMO Gopalgonj/Khal/P1/Design & XL File/Barshapara Khal/Input Output
Barshapara_khal_Dataprep_Input.xlsx
D:/Design_All_DC_5/AEMostofa/Southwest Project/SMO Gopalgonj/Khal/P1/Design & XL File/Barshapara Khal/Input Output


In [14]:
#inthis cell wefind direction of datamatrix

In [15]:

#settingpup log files
log_file_path=os.path.join(dir_name,"Khal_Input_Prep.log")
logger=logging.getLogger(__name__)
logger.setLevel("DEBUG")
fileHandler=logging.FileHandler(filename=log_file_path,mode="w")
#formatter=logging.Formatter('%(asctime)s - %(name)s - %(lineno)s - %(levelname)s - %(message)s')
formatter=logging.Formatter(' %(message)s')
fileHandler.setFormatter(formatter)
fileHandler.setLevel(logging.DEBUG)
logger.addHandler(fileHandler)
#formatter=logging.Formatter("{asctime}-{levelname}-{message}", style="{",datefmt="%Y-%m-%d %H:%M %S")
#testing logger Setup........................
logger.debug("Sucessfully Setup Logger for this programme.............")
#creating a second log file path and handler for handling individual function
log_file_path_function=os.path.join(dir_name,"function_log.log")
fileHandler_function=logging.FileHandler(filename=log_file_path_function,mode="w")
fileHandler_function.setFormatter(formatter)
fileHandler_function.setLevel(logging.DEBUG)

                                    

In [16]:
wb=opl.load_workbook(work_book_path)
print(wb.sheetnames)
sheetName="Package_Info"
matrix_df=pd.read_excel(work_book_path,sheet_name=sheetName)
logger.debug("Printing Data Matrix Direction.................\n")
logger.debug(matrix_df)
MAJOR_DATA_DIRECTION=matrix_df.iloc[1,1]
LONG_SLOPE_DIRECTION=matrix_df.iloc[2,1]
print("Data Direction={}".format(MAJOR_DATA_DIRECTION))
print("Long Slope direction=".format(LONG_SLOPE_DIRECTION))

['Raw_Cross_Section_Data', 'Package_Info', 'Data_index', 'Khal_Info', 'Notes_DC8', 'Headers', 'Лист1', 'Notes']
Data Direction=V
Long Slope direction=


In [17]:

index_wshhet=wb["Data_index"]
data_sheet=wb['Raw_Cross_Section_Data']
myframes=[]
mynames=[]

In [18]:
sheetName="Data_index"
index_df=pd.read_excel(work_book_path,sheet_name=sheetName)
index_df
"""Adding index data frame for writing"""
myframes.append(index_df)
mynames.append("index_df")

In [19]:
""""This Reads Data of Cross From segregated Sheet and Create a Data Frame """
logger.debug("\n\n###############################################\n\n")
logger.debug(index_df)
distance=[]
rl=[]
remarks=[]
chaianges=[]
sectionnos=[]
for index,row in index_df.iterrows():
    
    chaiange=row['chainage']
    section_no=row['XsectionNo']
    logger.debug("--------Intiated Data Transfer for section no={}--------\n\n\n\n" .format(section_no))
    r1=row['r1']
    r2=row['r2']
    c1=row['c1']
    c2=row['c2']
    direction=row['Direction']

    logger.debug("chainage={} sec={} row1={} row2={} col1={} col2={} {}".format(chaiange, section_no,
                                                                     r1,r2,c1,c2,direction))
    #myvalues=readSheetValue(data_sheet,r1,r2,c1,c2,chaiange,section_no)
    if(MAJOR_DATA_DIRECTION=="H"):
        myvalues=readSheetValue_Horizontal(data_sheet,r1,r2,c1,c2,chaiange,section_no,direction)
    else:
        myvalues=readSheetValue_Vertical(data_sheet,r1,r2,c1,c2,chaiange,section_no,direction)
    
    rl=rl+myvalues["rl"]
    distance=distance+myvalues['dist']
    remarks=remarks+myvalues["remarks"]
    chaianges=chaianges+myvalues["chainage"]
    sectionnos=sectionnos+myvalues["sectionNo"]
    logger.debug("--------Finished Data Transfer for section no={}--------\n\n\n\n" .format(section_no))
data={"Section No":sectionnos,"Chainage (m)":chaianges,"Distance (m)":distance,
      "RL (m)":rl,"Remarks":remarks}
output_frame=pd.DataFrame(data)
combined_survey_df=output_frame.copy(deep=True)

"""Adding index data frame for writing"""
myframes.append(output_frame)
mynames.append("Combined")
#output_frame.to_excel(out_path)
saveDataFrame(myframes,out_path2,mynames)   

row1=5 row2=17 col1=B col2=D
row1=5 row2=17 col1=2 col2=3
{'dist': [0, 5, 10, 11, 13, 14, 15, 16, 18, 19, 20, 25, 30], 'rl': [0.377, 0.372, 0.367, -0.038, -0.284, -0.485, -0.533, -0.484, -0.285, 0.033, 0.416, 0.422, 0.426], 'remarks': ['Open land', '', 'LB', '', '', '', 'CL', '', '', '', 'RB', '', 'Paddy land'], 'chainage': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'sectionNo': ['BPK0', 'BPK0', 'BPK0', 'BPK0', 'BPK0', 'BPK0', 'BPK0', 'BPK0', 'BPK0', 'BPK0', 'BPK0', 'BPK0', 'BPK0']}
row1=21 row2=33 col1=B col2=D
row1=21 row2=33 col1=2 col2=3
{'dist': [0, 5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 25, 30], 'rl': [0.457, 0.452, 0.447, -0.038, -0.184, -0.411, -0.474, -0.414, -0.198, -0.033, 0.422, 0.427, 0.437], 'remarks': ['Paddy land', '', 'LB', '', '', '', 'CL', '', '', '', 'RB', '', 'Paddy land'], 'chainage': [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], 'sectionNo': ['BPK1', 'BPK1', 'BPK1', 'BPK1', 'BPK1', 'BPK1', 'BP

In [20]:
"""Class for Creating Khal_Sheet_Index"""
class khal_sheet_Nos:
    def __init__(self,khal_Name,length,xsection_list,start_sheet_no,y0,dwg_base_no,dwg_date):
        self.Name=khal_Name
        self.y0=y0
        self.Index_map_sheet_No=start_sheet_no
        self.Titles=[]        
        self.Notes_Sheet_No=start_sheet_no+1
        no_long_section=math.ceil(length/3000.0)
        print("Total sheet for Long Section={}".format(no_long_section))
        self.Long_Section_Sheet_No=[start_sheet_no+1+i 
                                    for i in range(1,no_long_section+1)]
        print("LS Sheets={}".format(self.Long_Section_Sheet_No))
        k=self.Long_Section_Sheet_No[-1]
        cross_section_sheet_list=[]
        for index,elem in enumerate(xsection_list):
            if index%2==0:
                k=k+1
                cross_section_sheet_list.append(k)
            else:
                cross_section_sheet_list.append(k)
        data={"SectionName":xsection_list,"Sheet_No":cross_section_sheet_list}
        self.xsection_sheet_df=pd.DataFrame(data)
        self.Details=cross_section_sheet_list[-1]+1
        self.Next_Index=self.Details+1
        """Generating Titles for Drawing Sheet"""
        self.generateTitles()
        """Generating Sheet Coorinates"""
        n=len(self.Titles)
        self.Xcoord=[]
        self.Ycoord=[]
        self.width=[]
        self.DrawingNo=[]
        self.DrawingDate=[]
        self.DSSheetNo=[]
        #dwg_base_no,dwg_date
        for i in range(0,n):
            self.Xcoord.append(100000+i*10000)            
            self.Ycoord.append(self.y0)
            self.width.append(8000)
            if i+1 <=9:
                self.DrawingNo.append(dwg_base_no+"-0"+str(i+1)+"/"+str(n))
                self.DSSheetNo.append("0"+str(i+1)+"/"+str(n))
            else:
                self.DrawingNo.append(dwg_base_no+"-"+str(i+1)+"/"+str(n))
                self.DSSheetNo.append(str(i+1)+"/"+str(n))
            self.DrawingDate.append(dwg_date)
                
                
            
        
    def generateTitles(self):
        """generatig sheet Titles"""
        Titles=[]
        Titles.append("Index Map & Site Map for "+self.Name)
        Titles.append("Notes for "+self.Name)
        for x in self.Long_Section_Sheet_No:
            Titles.append("Long Section of "+self.Name)
        unique_xs_sheet=self.xsection_sheet_df.Sheet_No.unique()  
        for s in unique_xs_sheet:
            Titles.append("Typical Cross Section of "+self.Name)
        Titles.append("Details for "+self.Name)
        self.Titles=Titles
    
    
    
    def display(self):
        print("Khal Name={} Index Map={} Notes={} "              
              .format(self.Name,self.Index_map_sheet_No,self.Notes_Sheet_No))
        print("Long Sections={}".format(self.Long_Section_Sheet_No))
        print("Xsections={}".format(self.xsection_sheet_df))
        print("Details={}".format(self.Details))
"""Class for creating Khal Long Sections"""
class Khal_Long_Sections:
    def __init__(self,khal_name,xsection_list,index_df,LS_Code,head=None):
        self.xsection_codes=xsection_list
        if head:
            self.xsection_codes.insert(0,head)
        self.LS_codes=[LS_Code for xs in self.xsection_codes]
        data={"LSCode":self.LS_codes,"CSCode":self.xsection_codes}
        self.DataDetails=pd.DataFrame(data)

In [21]:
#saveDataFrame(myframes,out_path,mynames)

In [22]:
#output_frame

In [23]:
sheetName="Khal_Info"
khal_df=pd.read_excel(work_book_path,sheet_name=sheetName)
logger.debug("Begin Building Xsection Data Sheet................")
logger.debug("Printing Input Data Frame.............")
section_no=[]
chaiange=[]
center_line=[]
left_slope=[]
right_slope=[]
design_level=[]
bed_width=[]
start_chn=[]
end_chn=[]
sec_chn=[]
app_start_rl=[]
app_end_rl=[]
design_bed_rl=[]
sl_no=[]
ds_no=[]
cl_x=[]
drawing_loc=[]
SectionName=[]
left_bank_rl=[]
right_bank_rl=[]
cl_existing_rl=[]
cl_design_rl=[]
long_section_code=[]
long_section_chainage=[]


for index,row in khal_df.iterrows():
    kname=row['KhalName']
    n=row['m']
    B=row['B']
    Kcode=row['Khal_Code_Name']
    out_fall_rl=row['Ouftfall']
    long_slope=row['long_slope']
    mydata=index_df[index_df['KhalName']==kname]
    mysec_no=list(mydata['XsectionNo'])
    print("list of XSection for {}".format(kname))
    print(mysec_no)
    my_chn_in_meter=list(mydata['chainage'])
    section_no=section_no+ mysec_no
    my_left_slope=[n for x in mysec_no]
    left_slope=left_slope+ my_left_slope
    right_slope=right_slope+my_left_slope
    my_bottom_width=[B for x in mysec_no]
    bed_width= bed_width+my_bottom_width
    my_sec_chn=list(mydata['chainage'])
    #print(my_sec_chn)
    my_st_chn=calculteStartChainage(my_sec_chn)
    start_chn=start_chn+ my_st_chn
    my_end_chn=calculteEndChainage(my_sec_chn)
    end_chn=end_chn+my_end_chn
    
    if LONG_SLOPE_DIRECTION==1:
        my_start_rl=calcualteRLFromChainage(out_fall_rl,long_slope,my_st_chn)
    else:
        my_start_rl=calcualteRLFromChainageFromUS(out_fall_rl,long_slope,my_st_chn)
    app_start_rl=app_start_rl+my_start_rl
    
    if LONG_SLOPE_DIRECTION==1:
        my_end_rl=calcualteRLFromChainage(out_fall_rl,long_slope,my_end_chn)
    else:
        my_end_rl=calcualteRLFromChainageFromUS(out_fall_rl,long_slope,my_end_chn)
    app_end_rl=app_end_rl+my_end_rl
    
    my_sec_chn2=[x/1000 for x in my_sec_chn ]
    my_design_bed_rl=calcualteRLFromChainage(out_fall_rl,long_slope,my_sec_chn2)
    print("Printing design rl for {} khal".format(kname))
    print(my_design_bed_rl)
    design_bed_rl=design_bed_rl+my_design_bed_rl
    sec_chn=sec_chn+ my_sec_chn2
    print("section no...........")
    print(mysec_no)
    my_return_values=calcualte_cl_X(mysec_no,output_frame)
    my_cl_x=my_return_values["cl_x"]
    cl_x=cl_x+my_cl_x
    my_khal_sec_name=["Typical Cross Section of "+kname for s in mysec_no]
    SectionName=SectionName+my_khal_sec_name
    my_sec_loc=my_return_values["sec_loc"]
    drawing_loc=drawing_loc+my_sec_loc
    """Extracting Data for Khal Long Section"""
 
   
    my_left_bank_rl=my_return_values["lbank_rl"]
    left_bank_rl=left_bank_rl+my_left_bank_rl
    my_right_bank_rl=my_return_values["rabnk_rl"]
    right_bank_rl=right_bank_rl+my_right_bank_rl
    my_cl_rl=my_return_values["cl_rl"]
    cl_existing_rl=cl_existing_rl+ my_cl_rl
    cl_design_rl=cl_design_rl+my_design_bed_rl
    my_ls_code=["LS"+ Kcode for x in  mysec_no]
    long_section_code=long_section_code+my_ls_code
    my_long_sec_chn=my_return_values["ls_chaiange"]
    long_section_chainage=long_section_chainage+my_long_sec_chn
    #genrating data for drawing 
    print("Finished calculating {}".format(kname))

    
#print(mydata)
print(section_no)
print(left_slope)
print(right_slope)
print(bed_width)
print(start_chn)
print(end_chn)
print(app_start_rl)
print( app_end_rl)
print( design_bed_rl)
print("\n\n printing CL distance \n\n")
print(cl_x)
#sl	section_id	Design Bed Level	Double CL_X	Bottom Width	Left Slope	
#Right Slope	drawing_sheet	Location	Section Name	Chainage	
#start_point_app	end_point_app	start_rl	end_rl
sl_no=[i+1 for i in range(0,len(section_no))]
ds_no=[1 for i in range(0,len(section_no))]

#drawing_loc=[1 for i in range(0,len(section_no))]

"""Bulinging Data Frame For Xsection Data """  

data_xsec={"sl":sl_no,"section_id":section_no,"Design Bed Level":design_bed_rl,"CL_X":cl_x,
     "Bottom Width":bed_width,"Left Slope":left_slope,"Right Slope":right_slope,
     "drawing_sheet":ds_no,"Location":drawing_loc,"Section Name":SectionName,"Chainage":sec_chn,
      "start_point_app":start_chn,"end_point_app":end_chn,
    "start_rl":app_start_rl,"end_rl": app_end_rl }

"""
data={"sl":sl_no,"section_id":section_no,    
     "drawing_sheet":ds_no,"Location":drawing_loc,"Section Name":SectionName,"Chainage":sec_chn,
      "start_point_app":start_chn,"end_point_app":end_chn,
     }

"""

check_array_length_for_df(data_xsec)
xsection_df=pd.DataFrame(data_xsec)
#myframes.append(xsection_df)
#mynames.append("Xsection")
logger.debug("Finished Building Xsection Data Sheet................")

list of XSection for Barsha para khal
['BPK0', 'BPK1', 'BPK2', 'BPK3', 'BPK4', 'BPK5', 'BPK6', 'BPK7', 'BPK8', 'BPK9', 'BPK10', 'BPK11', 'BPK12', 'BPK13', 'BPK14', 'BPK15', 'BPK16', 'BPK17', 'BPK18', 'BPK19', 'BPK20']
Printing design rl for Barsha para khal khal
[-1.3, -1.3, -1.29, -1.29, -1.28, -1.28, -1.27, -1.27, -1.26, -1.26, -1.25, -1.25, -1.24, -1.24, -1.23, -1.23, -1.22, -1.22, -1.21, -1.21, -1.3]
section no...........
['BPK0', 'BPK1', 'BPK2', 'BPK3', 'BPK4', 'BPK5', 'BPK6', 'BPK7', 'BPK8', 'BPK9', 'BPK10', 'BPK11', 'BPK12', 'BPK13', 'BPK14', 'BPK15', 'BPK16', 'BPK17', 'BPK18', 'BPK19', 'BPK20']
Finished calculating Barsha para khal
['BPK0', 'BPK1', 'BPK2', 'BPK3', 'BPK4', 'BPK5', 'BPK6', 'BPK7', 'BPK8', 'BPK9', 'BPK10', 'BPK11', 'BPK12', 'BPK13', 'BPK14', 'BPK15', 'BPK16', 'BPK17', 'BPK18', 'BPK19', 'BPK20']
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [24]:
"""Bulinging Data Frame For Xsection Data"""  
data={"SectionNo":long_section_code, "Chainage":long_section_chainage,"CL_RL":cl_existing_rl,"RB_RL":right_bank_rl,
      "LB_RL":left_bank_rl,"DL":design_bed_rl}
long_section_df=pd.DataFrame(data)
myframes.append(long_section_df)
mynames.append("LS")
"""Building Data Frame For Section Diagnosis"""
#XSection No	Chainage (m)	CL_X	Left_Slope	Right_Slope	Design_Bed_Level	Bottom_Width
data={"XSection No":section_no,"Chainage (m)":long_section_chainage,"CL_X":cl_x,
      "Left_Slope":left_slope,"Right_Slope":right_slope,"Design_Bed_Level":design_bed_rl,
      "Bottom_Width":bed_width}
cross_section_list_df=pd.DataFrame(data)
myframes.append(cross_section_list_df)
mynames.append("Crossection_List")

In [25]:
#creating input path for files
#fname_combined_input_all=os.path.join(dir_name,"combined_input_all.xlsx")
#shutil.copy("Blank_Input_File_Khal.xlsx",fname_combined_input_all)
#shutil.copy("Blank_Input_File_Khal.xlsx",fname_combined_input_filtered)
#shutil.copy("Blank_Input_File_Khal.xlsx",fname_combined_input_1km)

In [26]:
xsection_df

,sl,section_id,Design Bed Level,CL_X,Bottom Width,Left Slope,Right Slope,drawing_sheet,Location,Section Name,Chainage,start_point_app,end_point_app,start_rl,end_rl
0,1,BPK0,-1.30,15.0,3,2,2,1,A,Typical Cross Section of Barsha para khal,0.00000,0.00000,0.05000,-1.30,-1.30
1,2,BPK1,-1.30,14.0,3,2,2,1,B,Typical Cross Section of Barsha para khal,0.10000,0.05000,0.15000,-1.30,-1.29
2,3,BPK2,-1.29,14.0,3,2,2,1,A,Typical Cross Section of Barsha para khal,0.20000,0.15000,0.25000,-1.29,-1.29
3,4,BPK3,-1.29,14.0,3,2,2,1,B,Typical Cross Section of Barsha para khal,0.30000,0.25000,0.35000,-1.29,-1.28
4,5,BPK4,-1.28,14.0,3,2,2,1,A,Typical Cross Section of Barsha para khal,0.40000,0.35000,0.45000,-1.28,-1.28
5,6,BPK5,-1.28,10.0,3,2,2,1,B,Typical Cross Section of Barsha para khal,0.50000,0.45000,0.55000,-1.28,-1.27
6,7,BPK6,-1.27,6.5,3,2,2,1,A,Typical Cross Section of Barsha para khal,0.60000,0.55000,0.65000,-1.27,-1.27
7,8,BPK7,-1.27,6.5,3,2,2,1,B,Typical Cross Section of Barsha para khal,0.70000,0.65000,0.75000,-1.27,-1.26
8,9,BPK8,-1.26,14.0,3,2,2,1,A,Typical Cross Section of Barsha para khal,0.80000,0.75000,0.85000,-1.26,-1.26
9,10,BPK9,-1.26,15.0,3,2,2,1,B,Typical Cross Section of Barsha para khal,0.90000,0.85000,0.95000,-1.26,-1.25


In [27]:
Title=[]
DesignNo=[]
design_date=[]
x0=[]
y0=[]
width=[]
sheet_no=[]
for index,row in khal_df.iterrows():
    kname=row['KhalName']
    dwg_no=row["DrawingNo"]
    mydate=row["Date"]
    print(mydate)
    print(type(mydate))
    #date = pd.to_datetime(mydate).dt.date
    date=mydate
    print("Khal={} drawing_no={} date={}".format(kname,dwg_no,date))
    myretun_values=calcualteDrawingSheetData(kname,dwg_no,date)
    Title=Title+myretun_values["Title"]
    DesignNo= DesignNo+myretun_values[ "DesignNo"]
    design_date=design_date+myretun_values["design_date"]
    x0=x0+myretun_values["x0"]
    y0=y0+myretun_values["y0"]
    width=width+myretun_values["width"]
    sheet_no=sheet_no+myretun_values["sheet_no"]
#Title	Design No	Date	x0	y0	width	sheet_no
 
data={"Title":Title,"Design No":DesignNo,"Date":design_date,"x0":x0,
      "y0":y0,"width":width,"sheet_no":sheet_no}


drawing_sheet_df=pd.DataFrame(data)
myframes.append(drawing_sheet_df)
mynames.append("Drawing_Sheet")


29/11/2024
<class 'str'>
Khal=Barsha para khal drawing_no=DC5-0000 date=29/11/2024
['BPK0', 'BPK1', 'BPK2', 'BPK3', 'BPK4', 'BPK5', 'BPK6', 'BPK7', 'BPK8', 'BPK9', 'BPK10', 'BPK11', 'BPK12', 'BPK13', 'BPK14', 'BPK15', 'BPK16', 'BPK17', 'BPK18', 'BPK19', 'BPK20']
[0.0, 100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0, 1500.0, 1600.0, 1700.0, 1800.0, 1900.0, 2.02]
length of Khal=2.02 Long_sec_sheet=1
total cross section=21 sheet_no=11
['Index Map & Site Map for Barsha para khal', 'Notes', 'Long Section of Barsha para khal', 'Typical Cross Section of Barsha para khal', 'Typical Cross Section of Barsha para khal', 'Typical Cross Section of Barsha para khal', 'Typical Cross Section of Barsha para khal', 'Typical Cross Section of Barsha para khal', 'Typical Cross Section of Barsha para khal', 'Typical Cross Section of Barsha para khal', 'Typical Cross Section of Barsha para khal', 'Typical Cross Section of Barsha para khal', 'Typical Cross

In [28]:
combined_survey_df

,Section No,Chainage (m),Distance (m),RL (m),Remarks
0,BPK0,0.00,0.0,0.377,Open land
1,BPK0,0.00,5.0,0.372,
2,BPK0,0.00,10.0,0.367,LB
3,BPK0,0.00,11.0,-0.038,
4,BPK0,0.00,13.0,-0.284,
...,...,...,...,...,...
287,BPK20,2.02,13.0,-0.241,
288,BPK20,2.02,13.5,-0.180,
289,BPK20,2.02,14.0,0.030,RB
290,BPK20,2.02,20.0,0.035,


In [29]:
"""Modifying Sheet Number in Xsection Df"""
khals=index_df['KhalName'].unique()
total_sheet_nos=[]
print(khals)
start_sheet_no=3
start_sheet=1
khal_drawing_sheets=[]
y0=0
for khal in khals:
    #print(khal)
    xsections=list(index_df[index_df['KhalName']==khal]["XsectionNo"])
    chaianage=list(index_df[index_df['KhalName']==khal]["chainage"])
    length=chaianage[-1]-chaianage[0]  
    """Retrieving Drawing No and Drawing Date from Khal_df"""
    idx=khal_df.index[khal_df['KhalName']==khal]
    dwg_base_no=khal_df.iloc[idx,6].values[0]
    #dwg_date=khal_df.iloc[idx,7].dt.date.values[0]
    dwg_date=khal_df.iloc[idx,7].values[0]
    print("drawing date={}".format(dwg_date))
    #print(math.ceil(length/5000.0))
    #self,length,xsection_list,start_sheet_no
    current_khal_ds=khal_sheet_Nos(khal,length,xsections,start_sheet,y0,dwg_base_no,dwg_date)
    section_list_df=current_khal_ds.xsection_sheet_df
    #print(section_list_df)
    start_sheet=current_khal_ds.Next_Index
    current_khal_ds.display()
    khal_drawing_sheets.append(current_khal_ds)
    y0=y0+10000

['Barsha para khal']
drawing date=29/11/2024
Total sheet for Long Section=1
LS Sheets=[3]
Khal Name=Barsha para khal Index Map=1 Notes=2 
Long Sections=[3]
Xsections=   SectionName  Sheet_No
0         BPK0         4
1         BPK1         4
2         BPK2         5
3         BPK3         5
4         BPK4         6
5         BPK5         6
6         BPK6         7
7         BPK7         7
8         BPK8         8
9         BPK9         8
10       BPK10         9
11       BPK11         9
12       BPK12        10
13       BPK13        10
14       BPK14        11
15       BPK15        11
16       BPK16        12
17       BPK17        12
18       BPK18        13
19       BPK19        13
20       BPK20        14
Details=15


In [30]:
""" Saving All the Data Frame"""
saveDataFrame(myframes,out_path,mynames)

In [31]:
sheetName="Package_Info"

auxilary_info_df=pd.read_excel(work_book_path,sheet_name=sheetName)
base_acad_input_name=auxilary_info_df.iloc[0,1]
"""Crtaing Input Excel File Path for Autocad Input"""
fname_combined_input_all=os.path.join(dir_name,"Combined_input_"+
                                  base_acad_input_name+"_all"+"."+file_names[1])
fname_combined_input_filtered=os.path.join(dir_name,"Combined_input_"+
                                  base_acad_input_name+"_filtered"+"."+file_names[1])
fname_combined_input_1km=os.path.join(dir_name,"Combined_input_"+
                                  base_acad_input_name+"_1km"+"."+file_names[1])

#copying Files to Destination
shutil.copy("Blank_Input_File_Khal.xlsx",fname_combined_input_all)
shutil.copy("Blank_Input_File_Khal.xlsx",fname_combined_input_filtered)
shutil.copy("Blank_Input_File_Khal.xlsx",fname_combined_input_1km)





'D:/Design_All_DC_5/AEMostofa/Southwest Project/SMO Gopalgonj/Khal/P1/Design & XL File/Barshapara Khal/Input Output\\Combined_input_TP_KEX_10_14_1km.xlsx'

In [32]:
df_shape=auxilary_info_df.shape
print(df_shape)
"Populating combined_input_all"
writeDataFrameToExcel(xsection_df,2,1,fname_combined_input_all,"Xsection")
writeDataFrameToExcel(output_frame,2,2,fname_combined_input_all,"Combined")
writeDataFrameToExcel(cross_section_list_df,2,2,fname_combined_input_all,"Crossection_List")
writeDataFrameToExcel(long_section_df,2,1,fname_combined_input_all,"LS")
#writeDataFrameToExcel(drawing_sheet_df,2,2,fname_combined_input_all,"Drawing_Sheet")

(3, 3)
(21, 15)
(292, 5)
(21, 7)
(21, 6)


In [33]:
"""Preparing Long Section Data"""
long_sections_details=[]
LS_Codes=[]
section_list_for_longsection=[]
cl_RL_LS=[]
chaiage_LS=[]
lb_RL_LS=[]
rb_RL_LS=[]
dl_RL_LS=[]
for khal in khals:
    #print(khal)
    xsections=list(index_df[index_df['KhalName']==khal]["XsectionNo"])
    chaianage=list(index_df[index_df['KhalName']==khal]["chainage"])
    khal_code=list(khal_df[khal_df['KhalName']==khal]["Khal_Code_Name"])[0]
    ls_base_code="LS"+khal_code
    
    #print("Khal code={} Long Section Base code={}".format(khal_code,ls_base_code))
    length=chaianage[-1]-chaianage[0]
    n=math.ceil(length/3000)
    parts=np.array_split(xsections,n)
    LSCODE=ls_base_code+str(1)
    current_LS=Khal_Long_Sections(khal,list(parts[0]),index_df,LSCODE,head=None)
    long_sections_details.append(current_LS)    
    for i in range(1,n): 
        LSCODE=ls_base_code+str(i+1)
          
        current_LS=Khal_Long_Sections(khal,list(parts[i]),index_df,LSCODE,
                                      head=parts[i-1][-1])
       
        long_sections_details.append(current_LS)
        
for ls in long_sections_details:
    #print(ls.DataDetails
    LS_Codes=LS_Codes+list(ls.DataDetails['LSCode'])
    section_list_for_longsection=section_list_for_longsection+list(ls.DataDetails['CSCode'])   

    #calcualte_cl_X(sec_list,combined_df)
    
#print(LS_Codes)
#print( section_list_for_longsection)
myreturn_values=calcualte_cl_X(section_list_for_longsection,output_frame)
#print(myreturn_values)
cl_RL_LS=myreturn_values['cl_rl']
lb_RL_LS=myreturn_values['lbank_rl']
rb_RL_LS=myreturn_values['rabnk_rl']
chaiage_LS=myreturn_values['ls_chaiange']
for xsec in section_list_for_longsection:
    rindex=xsection_df.index[xsection_df['section_id']==xsec]
    rl=xsection_df.iloc[rindex,2].values[0]
    #print("xsection={} rl={}".format(xsec,rl))
    dl_RL_LS.append(rl)
data={"SectionNo":LS_Codes,"Chainage":chaiage_LS,
      "CL_RL":cl_RL_LS,"RB_RL":rb_RL_LS,"LB_RL":lb_RL_LS,"DL":dl_RL_LS}
LS_df=pd.DataFrame(data)
writeDataFrameToExcel(LS_df,2,1,fname_combined_input_all,"LS")
"""Preparing Long Section Drawing"""
ls_drawing_sheets=[]
ls_start_chn=[]
ls_finish_chn=[]
ls_section_name=[]
lsection_unique_codes=LS_df.SectionNo.unique()
print(lsection_unique_codes)
notes_start_index=[]
notes_finish_index=[]
cross_section_sheet_loc=[]
"""Variables for Cross Section Sheet"""
XS_Sheet_Titles=[]
XS_Sheet_Xorigin=[]
XS_Sheet_Yorigin=[]
XS_Sheet_width=[]
XS_design_no=[]
XS_design_date=[]
XS_drawing_sheet_no=[]
for kds in khal_drawing_sheets:
    ls_drawing_sheets=ls_drawing_sheets+kds.Long_Section_Sheet_No
    notes_start_index.append(kds.Notes_Sheet_No)
    notes_finish_index.append(kds.Notes_Sheet_No)
    sheet_loc=list(kds.xsection_sheet_df["Sheet_No"])
    cross_section_sheet_loc=cross_section_sheet_loc+ sheet_loc
    for x in kds.Long_Section_Sheet_No:
        lsn="Long Section of "+kds.Name
        ls_section_name.append(lsn)
    print(ls_drawing_sheets)
    """Extracting Drawing Sheet Data"""
  
    XS_Sheet_Titles=XS_Sheet_Titles+kds.Titles
    XS_Sheet_Xorigin=XS_Sheet_Xorigin+kds.Xcoord
    XS_Sheet_Yorigin=XS_Sheet_Yorigin+kds.Ycoord
    XS_Sheet_width=XS_Sheet_width+kds.width
    XS_design_no=XS_design_no+kds.DrawingNo
    XS_design_date=XS_design_date+kds.DrawingDate
    XS_drawing_sheet_no=XS_drawing_sheet_no+kds.DSSheetNo
for lcode in lsection_unique_codes:
    chn_list=LS_df[LS_df["SectionNo"]==lcode].Chainage
    ls_start_chn.append(min(chn_list)/1000)
    ls_finish_chn.append(max(chn_list)/1000)
print(ls_start_chn)
print(ls_finish_chn)
print(ls_section_name)
#sl	SectionId	drawing_sheet	Secion Name	Start	End

data={"SectionId":lsection_unique_codes,
      "drawing_sheet":ls_drawing_sheets,
      "Secion Name":ls_section_name,
      "Start":ls_start_chn,"End":ls_finish_chn}
ls_drawing_sheet_df=pd.DataFrame(data)
writeDataFrameToExcel(ls_drawing_sheet_df,2,2,fname_combined_input_all,"LSection")
"""Notes Location"""
#Start_index	Finish_index

data={"Start_index":notes_start_index,"Finish_index":notes_finish_index}
notes_loc_df=pd.DataFrame(data)
writeDataFrameToExcel(notes_loc_df,2,1,fname_combined_input_all,"Notes_Location")
print(cross_section_sheet_loc)
xsection_df["drawing_sheet"]=cross_section_sheet_loc
writeDataFrameToExcel(xsection_df,2,1,fname_combined_input_all,"Xsection")
"""Generating Data Frame for Drawing Sheet"""
#Title	Design No	Date	x0	y0	width	sheet_no

data={"Title":XS_Sheet_Titles,"Design No":XS_design_no,"Date":XS_design_date,
      "x0":XS_Sheet_Xorigin,
      "y0":XS_Sheet_Yorigin,"width":XS_Sheet_width,
      "sheet_no":XS_drawing_sheet_no
     }
drawing_sheeet_df2=pd.DataFrame(data)
writeDataFrameToExcel(drawing_sheeet_df2,2,2,fname_combined_input_all,"Drawing_Sheet")

(21, 6)
['LSBPK1']
[3]
[0.0]
[1.9]
['Long Section of Barsha para khal']
(1, 5)
(1, 2)
[4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14]
(21, 15)
(15, 7)


In [34]:
"""Transferring notes and Header Data"""
sheetName="Notes"
notes_df=pd.read_excel(work_book_path,sheet_name=sheetName)
notes_df
writeDataFrameToExcel(notes_df,2,1,fname_combined_input_all,"Notes")
sheetName="Headers"
headers_df=pd.read_excel(work_book_path,sheet_name=sheetName)
headers_df
writeDataFrameToExcel(headers_df,2,1,fname_combined_input_all,"Headers")

(28, 8)
(9, 2)


In [35]:
output_frame

,Section No,Chainage (m),Distance (m),RL (m),Remarks
0,BPK0,0.00,0.0,0.377,Open land
1,BPK0,0.00,5.0,0.372,
2,BPK0,0.00,10.0,0.367,LB
3,BPK0,0.00,11.0,-0.038,
4,BPK0,0.00,13.0,-0.284,
...,...,...,...,...,...
287,BPK20,2.02,13.0,-0.241,
288,BPK20,2.02,13.5,-0.180,
289,BPK20,2.02,14.0,0.030,RB
290,BPK20,2.02,20.0,0.035,
